In [ ]:
! pip install -U datasets "langchain<0.2.0,>=0.1.52" "langchain-core<0.2.0,>=0.1.52" langchain-mongodb langchain-openai pymongo pandas "rag-conversation<0.2.0,>=0.1.0" "langchain-text-splitters<0.1.0,>=0.0.1"

In [2]:
import getpass
OPENAI_API_KEY = getpass.getpass("Enter your OpenAI API key:")

In [3]:
MONGODB_URI = getpass.getpass("Enter your MongoDB connection string:")

In [ ]:
from datasets import load_dataset
import pandas as pd

data = load_dataset("MongoDB/embedded_movies")
df = pd.DataFrame(data["train"])

# Only keep records where the fullplot field is not null
df = df[df["fullplot"].notna()]

# Renaming the embedding field to "embedding" -- required by LangChain
df.rename(columns={"plot_embedding": "embedding"}, inplace=True)

In [4]:
from pymongo import MongoClient

# Initialize MongoDB python client
client = MongoClient(MONGODB_URI)

DB_NAME = "langchain_chatbot"
COLLECTION_NAME = "data"
ATLAS_VECTOR_SEARCH_INDEX_NAME = "vector_index"
collection = client[DB_NAME][COLLECTION_NAME]

In [5]:
# Delete any existing records in the collection
collection.delete_many({})

DeleteResult({'n': 7, 'electionId': ObjectId('7fffffff0000000000000191'), 'opTime': {'ts': Timestamp(1717607827, 36), 't': 401}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1717607827, 42), 'signature': {'hash': b'>h\x18H\x04\x1ae\xa5Y\xbad\x1a"\x0c\xdd\xad\xa2\xba\x16\xfb', 'keyId': 7337002600652341271}}, 'operationTime': Timestamp(1717607827, 36)}, acknowledged=True)

In [ ]:
# Data Ingestion
records = df.to_dict('records')
collection.insert_many(records)

print("Data ingestion into MongoDB completed")

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_mongodb import MongoDBAtlasVectorSearch

# Using the text-embedding-ada-002 since that's what was used to create embeddings in the movies dataset
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY, model="text-embedding-ada-002")

# Vector Store Creation
vector_store = MongoDBAtlasVectorSearch.from_connection_string(
    connection_string=MONGODB_URI,
    namespace=DB_NAME + "." + COLLECTION_NAME,
    embedding= embeddings,
    index_name=ATLAS_VECTOR_SEARCH_INDEX_NAME,
    text_key="fullplot"
)


In [ ]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [ ]:

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Generate context using the retriever, and pass the user question through
retrieve = {"context": retriever | (lambda docs: "\n\n".join([d.page_content for d in docs])), "question": RunnablePassthrough()}
template = """Answer the question based only on the following context: \
{context}

Question: {question}
"""
# Defining the chat prompt
prompt = ChatPromptTemplate.from_template(template)
# Defining the model to be used for chat completion
model = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
# Parse output as a string
parse_output = StrOutputParser()

# Naive RAG chain
naive_rag_chain = (
    retrieve
    | prompt
    | model
    | parse_output
)

In [ ]:
naive_rag_chain.invoke("What is the best movie to watch when sad?")

In [ ]:
! export LANGCHAIN_TRACING_V2=true
! export LANGCHAIN_API_KEY=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

In [ ]:
from langchain_mongodb.chat_message_histories import MongoDBChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import MessagesPlaceholder

def get_session_history(session_id: str) -> MongoDBChatMessageHistory:
        return MongoDBChatMessageHistory(MONGODB_URI, session_id, database_name=DB_NAME, collection_name="history")

In [ ]:
standalone_system_prompt = """
Given a chat history and a follow-up question, rephrase the follow-up question to be a standalone question. \
Do NOT answer the question, just reformulate it if needed, otherwise return it as is. \
Only return the final standalone question. \
"""
standalone_question_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", standalone_system_prompt),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

question_chain = standalone_question_prompt | model | parse_output

In [ ]:
retriever_chain = RunnablePassthrough.assign(context=question_chain | retriever | (lambda docs: "\n\n".join([d.page_content for d in docs])))

In [ ]:
rag_system_prompt = """Answer the question based only on the following context: \
{context}
"""
rag_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", rag_system_prompt),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

In [ ]:
# RAG chain
rag_chain = (
    retriever_chain
    | rag_prompt
    | model
    | parse_output
)

# RAG chain with history
with_message_history = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history",
)

In [ ]:
# with_message_history.invoke({"question": "What is the best movie to watch when sad?"}, {"configurable": {"session_id": "1"}})
# with_message_history.invoke({"question": "How about something more light?"}, {"configurable": {"session_id": "1"}})

In [ ]:
from langchain_mongodb.cache import MongoDBAtlasSemanticCache
from langchain_core.globals import set_llm_cache

set_llm_cache(MongoDBAtlasSemanticCache(
    connection_string=MONGODB_URI,
    embedding=embeddings,
    collection_name="semantic_cache",
    database_name=DB_NAME,
    index_name=ATLAS_VECTOR_SEARCH_INDEX_NAME,
    wait_until_ready=True # Optional, waits until the cache is ready to be used
))

In [ ]:
naive_rag_chain.invoke("What is the best movie to watch when sad?")